# **Perspectiva General de Mercado - DatAlpine**
*Filtro de tipo de mercado*

In [33]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
import numpy as np

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()
# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and 'alfa' in nombre and "queretaro" in nombre 
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_alfa_abril_2024_queretaro',
 'df_alfa_agosto_2024_queretaro',
 'df_alfa_marzo_2024_queretaro',
 'df_alfa_mayo_2024_queretaro']

In [34]:
# Iterar sobre cada DataFrame en la lista filtrada
for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame usando globals()
    df = globals()[nombre_df]
    df.rename(columns={'Categoria':'categoria','Category':'categoria','Propiedad':'propiedad','Precio':'precio','m_total':'metros_total','m2_total':'metros_total','m_construido':'metros_construido','m2_construido':'metros_construido','Estacionamientos':'estacionamiento','Recamaras':'recamaras','Banos':'baño','baños':'baño','Medio_banos':'medio_baño','Banos_Total':'baño_total','precio_m2_cons':'precio_m2_construido'}, inplace=True)
        # Asignar el DataFrame modificado de nuevo a la variable global
    globals()[nombre_df] = df 
print("Columnas renombradas en los DataFrames filtrados.")

Columnas renombradas en los DataFrames filtrados.


In [35]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['id','categoria','propiedad','precio','metros_total','metros_construido', 'precio_m2_construido','estacionamiento','recamaras', 'baño', 'medio_baño', 'baño_total']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)

In [36]:
df_total = pd.concat(dataframes_list, ignore_index=True)

In [37]:
df_total.columns

Index(['id', 'categoria', 'propiedad', 'precio', 'metros_total',
       'metros_construido', 'precio_m2_construido', 'estacionamiento',
       'recamaras', 'baño', 'medio_baño', 'baño_total'],
      dtype='object')

In [38]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38611 entries, 0 to 38610
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    38611 non-null  int64  
 1   categoria             38611 non-null  object 
 2   propiedad             38611 non-null  object 
 3   precio                38611 non-null  int64  
 4   metros_total          38611 non-null  int64  
 5   metros_construido     38611 non-null  int64  
 6   precio_m2_construido  38611 non-null  float64
 7   estacionamiento       38611 non-null  int64  
 8   recamaras             38611 non-null  int64  
 9   baño                  38611 non-null  int64  
 10  medio_baño            38611 non-null  int64  
 11  baño_total            38611 non-null  float64
dtypes: float64(2), int64(8), object(2)
memory usage: 3.5+ MB


In [39]:
rangos_precio = {
    "E1": (0, 500000),
    "E2": (500000, 750000),
    "E3": (750001, 1000000),
    "D1": (1000001, 1250000),
    "D2": (1250001, 1500000),
    "D3": (1500001, 1750000),
    "C1": (1750001, 2000000),
    "C2": (2000001, 2250000),
    "C3": (2250001, 2500000),
    "B1": (2500001, 2750000),  
    "B2": (2750001, 3000000),  
    "B3": (3000001, 3250000),  
    "A1": (3250001, 3500000),
    "A2": (3500001, 3750000),
    "A3": (3750001, 4000000),
    "S1": (4000001, 6000000),
    "S2": (6000001, 8000000),
    "S3": (8000001, 12000000),
    "L1": (12000001, 14000000),
    "L2": (14000001, 16000000),
    "L3": (16000001, 18000000),
    "L+": (18000001, 22000000),
    "ELITE": (22000001, float('inf'))
}

# Función para asignar la categoría según el precio
def asignar_categoria(precio):
    for categoria, (limite_inferior, limite_superior) in rangos_precio.items():
        if limite_inferior <= precio < limite_superior:
            return categoria
    return None  # En caso de que el precio no caiga en ningún rango (caso raro)

df_total = df_total.drop(columns=['categoria'])
df_total['categoria'] = df_total['precio'].apply(asignar_categoria)

In [40]:
import numpy as np 
condition = (df_total['baño_total'] % 1 == 0.5) # Valores que contienen .5    
# Convertir al número entero anterior
df_total.loc[condition, 'baño_total'] = np.floor(df_total['baño_total']) 
df_total['baño_total'] = df_total['baño_total'].clip(lower=1,upper=8) # Limitar los valores 
df_total['estacionamiento'] = df_total['estacionamiento'].clip(lower=1,) #upper=8
df_total['recamaras'] = df_total['recamaras'].clip(lower=1,)#upper=8 

- Casa promedio
- Casa Mediana
- Precio mercado

- m2 Terreno
- M2 Construcción
- Precio m2 construcción

# Casa promedio

In [41]:
df_total['segmento'] = df_total['categoria'].str[0]
grupo_promedios = round(df_total.groupby('segmento')[['estacionamiento', 'recamaras', 'baño_total']].mean(),0)
grupo_promedios.astype(int)
# Renombrar las columnas para mayor claridad
grupo_promedios.rename(columns={
    'estacionamiento': 'Estacionamientos',
    'recamaras': 'Recámaras',
    'baño_total': 'Baños',
}, inplace=True)

# Calcular el promedio de estacionamientos, recámaras y baños en general (sin segmento específico)
promedios_generales = round(df_total[['estacionamiento', 'recamaras', 'baño_total']].mean(),0)
promedios_generales = promedios_generales.astype(int)
# Renombrar las columnas para mayor claridad
promedios_generales.rename(index={
    'estacionamiento': 'Estacionamientos',
    'recamaras': 'Recámaras',
    'baño_total': 'Baños',
}, inplace=True)
# Agregar los promedios generales al dataframe de grupo_promedios
promedios_generales = pd.DataFrame(promedios_generales).T
promedios_generales['segmento'] = 'general'

casa_promedio = pd.concat([grupo_promedios, promedios_generales.set_index('segmento')], ignore_index=False)

print(grupo_promedios)
print(promedios_generales)
print(casa_promedio)

# Convertir el dataframe a JSON
grupo_promedios_json = casa_promedio.to_json(orient='index')

# Guardar el JSON en un archivo
with open('assets/tablas/grupo_casa_promedio.json', 'w') as f:
    f.write(grupo_promedios_json)

          Estacionamientos  Recámaras  Baños
segmento                                    
A                      2.0        3.0    2.0
B                      2.0        3.0    2.0
C                      2.0        3.0    2.0
D                      2.0        3.0    2.0
E                      2.0        3.0    2.0
L                      3.0        4.0    3.0
S                      2.0        3.0    3.0
   Estacionamientos  Recámaras  Baños segmento
0                 2          3      2  general
          Estacionamientos  Recámaras  Baños
segmento                                    
A                      2.0        3.0    2.0
B                      2.0        3.0    2.0
C                      2.0        3.0    2.0
D                      2.0        3.0    2.0
E                      2.0        3.0    2.0
L                      3.0        4.0    3.0
S                      2.0        3.0    3.0
general                2.0        3.0    2.0


# Casa Mediana

In [42]:
df_total['segmento'] = df_total['categoria'].str[0]
grupo_mediana = round(df_total.groupby('segmento')[['estacionamiento', 'recamaras', 'baño_total']].median(),0)
grupo_mediana.astype(int)
# Renombrar las columnas para mayor claridad
grupo_mediana.rename(columns={
    'estacionamiento': 'Estacionamientos',
    'recamaras': 'Recámaras',
    'baño_total': 'Baños',
}, inplace=True)
print(grupo_mediana)

# Calcular mediana de estacionamientos, recámaras y baños en general (sin segmento específico)
mediana_general = round(df_total[['estacionamiento', 'recamaras', 'baño_total']].median(),0)
mediana_general = mediana_general.astype(int)
# Renombrar las columnas para mayor claridad
mediana_general.rename(index={
    'estacionamiento': 'Estacionamientos',
    'recamaras': 'Recámaras',
    'baño_total': 'Baños',
}, inplace=True)
mediana_general = pd.DataFrame(mediana_general).T
mediana_general['segmento'] = 'general'
print(mediana_general)

casa_media = pd.concat([grupo_mediana, mediana_general.set_index('segmento')], ignore_index=False)
print(casa_media)

# Guardar el JSON en un archivo
grupo_promedios_json = casa_media.to_json(orient='index')
with open('assets/tablas/grupo_casa_mediana.json', 'w') as f:
    f.write(grupo_promedios_json)

          Estacionamientos  Recámaras  Baños
segmento                                    
A                      2.0        3.0    2.0
B                      2.0        3.0    2.0
C                      2.0        3.0    2.0
D                      2.0        3.0    2.0
E                      2.0        3.0    2.0
L                      2.0        4.0    2.0
S                      2.0        3.0    2.0
   Estacionamientos  Recámaras  Baños segmento
0                 2          3      2  general
          Estacionamientos  Recámaras  Baños
segmento                                    
A                      2.0        3.0    2.0
B                      2.0        3.0    2.0
C                      2.0        3.0    2.0
D                      2.0        3.0    2.0
E                      2.0        3.0    2.0
L                      2.0        4.0    2.0
S                      2.0        3.0    2.0
general                2.0        3.0    2.0


# Precio Mercado

In [43]:
grupo_descripcion = df_total.groupby('segmento')['precio'].describe().reset_index()
grupo_descripcion = grupo_descripcion[['segmento','min', 'max','mean','50%']] #Columnas deseadas
grupo_descripcion.rename(columns={
    '50%': 'Mediana',
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)
grupo_descripcion 

,segmento,Mínimo,Máximo,Promedio,Mediana
0,A,3251000.0,3.999999e+06,3.641822e+06,3650000.0
1,B,2521000.0,3.248000e+06,2.895003e+06,2900000.0
2,C,1752000.0,2.499000e+06,2.150071e+06,2175000.0
3,D,1003000.0,1.742360e+06,1.406566e+06,1406070.0
4,E,280000.0,9.680000e+09,1.990069e+07,880000.0
5,L,12220000.0,2.200000e+07,1.640307e+07,16450000.0
6,S,4003000.0,1.198000e+07,6.211986e+06,5680000.0


In [44]:
# Calculate the average of the 'Mínimo' column for all segments except 'general'
average_minimo = grupo_descripcion[grupo_descripcion['segmento'] != 'general']['Mínimo'].mean()
average_minimo

np.float64(3575714.285714286)

In [45]:
# Función para formatear valores en formato monetario
def formato_monetario(valor):
    return f"${valor:,.2f}"  # Formato con $ y 2 decimales, separador de miles

# Aplicar formato en m2 a los valores, excluyendo la columna 'segmento'
for i in range(1, 5):  # 1, 2, 3, 4 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    grupo_descripcion.iloc[:, i] = grupo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
grupo_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_5124\913539489.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     $3,251,000.00
1     $2,521,000.00
2     $1,752,000.00
3     $1,003,000.00
4       $280,000.00
5    $12,220,000.00
6     $4,003,000.00
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descripcion.iloc[:, i] = grupo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_5124\913539489.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0        $3,999,999.00
1        $3,248,000.00
2        $2,499,000.00
3        $1,742,360.00
4    $9,680,000,000.00
5       $21,999,999.00
6       $11,980,000.00
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype fir

,segmento,Mínimo,Máximo,Promedio,Mediana
0,A,"$3,251,000.00","$3,999,999.00","$3,641,821.90","$3,650,000.00"
1,B,"$2,521,000.00","$3,248,000.00","$2,895,003.13","$2,900,000.00"
2,C,"$1,752,000.00","$2,499,000.00","$2,150,071.26","$2,175,000.00"
3,D,"$1,003,000.00","$1,742,360.00","$1,406,565.67","$1,406,070.00"
4,E,"$280,000.00","$9,680,000,000.00","$19,900,688.75","$880,000.00"
5,L,"$12,220,000.00","$21,999,999.00","$16,403,074.51","$16,450,000.00"
6,S,"$4,003,000.00","$11,980,000.00","$6,211,986.10","$5,680,000.00"


In [50]:
todo_descripcion = df_total[(~df_total['metros_construido'].isin([0,1,2,3,4,5])) & (df_total['precio_m2_construido'] >= 280000)]['precio_m2_construido'].describe()[['min', 'max', 'mean', '50%']].to_frame().T
todo_descripcion['segmento'] = 'general'

todo_descripcion.rename(columns={
    '50%': 'Mediana',
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Aplicar formato monetario a los valores
for i in range(4):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)

todo_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_5124\2215839931.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio_m2_construido    $284,444.44
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_5124\2215839931.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio_m2_construido    $38,695,652.17
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_5124\2215839931.py:13: FutureWarning: Setting an item of incompatible dtype i

,Mínimo,Máximo,Promedio,Mediana,segmento
precio_m2_construido,"$284,444.44","$38,695,652.17","$12,556,748.61","$3,863,871.31",general


In [51]:
# Guardar el JSON en un archivo
resultados_finales_descripcion = pd.concat([grupo_descripcion, todo_descripcion], ignore_index=True)
resultados_finales_desc_json = resultados_finales_descripcion.to_json(orient='index')
with open('assets/tablas/grupo_precio_mercado.json', 'w') as f:
    f.write(resultados_finales_desc_json)

In [52]:
resultados_finales_descripcion

,segmento,Mínimo,Máximo,Promedio,Mediana
0,A,"$3,251,000.00","$3,999,999.00","$3,641,821.90","$3,650,000.00"
1,B,"$2,521,000.00","$3,248,000.00","$2,895,003.13","$2,900,000.00"
2,C,"$1,752,000.00","$2,499,000.00","$2,150,071.26","$2,175,000.00"
3,D,"$1,003,000.00","$1,742,360.00","$1,406,565.67","$1,406,070.00"
4,E,"$280,000.00","$9,680,000,000.00","$19,900,688.75","$880,000.00"
5,L,"$12,220,000.00","$21,999,999.00","$16,403,074.51","$16,450,000.00"
6,S,"$4,003,000.00","$11,980,000.00","$6,211,986.10","$5,680,000.00"
7,general,"$284,444.44","$38,695,652.17","$12,556,748.61","$3,863,871.31"


# M2 Terreno

In [53]:
df_total_terreno = df_total[~df_total['metros_total'].isin([0, 1, 2])]
df_descripcion = df_total_terreno.groupby('segmento')['metros_total'].describe().reset_index()
df_descripcion = df_descripcion[['segmento', 'min', 'max', 'mean', '50%']] #Columnas deseadas
df_descripcion.rename(columns={
    '50%': 'Mediana',
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)


# Función para formatear valores en formato metros
def formato_metros(valor):
    return f"{valor:,.2f} m²"

# Aplicar formato en m2 a los valores, excluyendo la columna 'segmento'
for i in range(1, 5):  # 1, 2, 3, 4 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    df_descripcion.iloc[:, i] = df_descripcion.iloc[:, i].astype(float).apply(formato_metros)
df_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_5124\2211586206.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     30.00 m²
1     14.00 m²
2     42.00 m²
3     44.00 m²
4     35.00 m²
5    103.00 m²
6     15.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_descripcion.iloc[:, i] = df_descripcion.iloc[:, i].astype(float).apply(formato_metros)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_5124\2211586206.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       22,415.00 m²
1        1,387.00 m²
2       11,267.00 m²
3        2,144.00 m²
4      290,000.00 m²
5       80,000.00 m²
6    2,500,000.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_descripcion.iloc[:, i] = df_descripcion.iloc[

,segmento,Mínimo,Máximo,Promedio,Mediana
0,A,30.00 m²,"22,415.00 m²",168.69 m²,145.00 m²
1,B,14.00 m²,"1,387.00 m²",129.65 m²,124.00 m²
2,C,42.00 m²,"11,267.00 m²",123.62 m²,110.00 m²
3,D,44.00 m²,"2,144.00 m²",113.94 m²,86.00 m²
4,E,35.00 m²,"290,000.00 m²",695.41 m²,237.00 m²
5,L,103.00 m²,"80,000.00 m²",930.54 m²,700.00 m²
6,S,15.00 m²,"2,500,000.00 m²",553.95 m²,250.00 m²


In [54]:
todo_descripcion = df_total[~df_total['metros_total'].isin([0, 1, 2])]['metros_total'].describe()[['min', 'max', 'mean', '50%']].to_frame().T
todo_descripcion['segmento'] = 'general'

todo_descripcion.rename(columns={
    '50%': 'Mediana',
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Aplicar formato monetario a los valores
for i in range(4):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_metros)

todo_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_5124\1308419453.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_total    14.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_metros)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_5124\1308419453.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_total    2,500,000.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_metros)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_5124\1308419453.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will ra

,Mínimo,Máximo,Promedio,Mediana,segmento
metros_total,14.00 m²,"2,500,000.00 m²",421.33 m²,189.00 m²,general


In [55]:
# Guardar el JSON en un archivo
resultados_finales_descripcion = pd.concat([df_descripcion, todo_descripcion], ignore_index=True)
resultados_finales_desc_json = resultados_finales_descripcion.to_json(orient='index')
with open('assets/tablas/grupo_terreno.json', 'w') as f:
    f.write(resultados_finales_desc_json)

In [56]:
resultados_finales_descripcion

,segmento,Mínimo,Máximo,Promedio,Mediana
0,A,30.00 m²,"22,415.00 m²",168.69 m²,145.00 m²
1,B,14.00 m²,"1,387.00 m²",129.65 m²,124.00 m²
2,C,42.00 m²,"11,267.00 m²",123.62 m²,110.00 m²
3,D,44.00 m²,"2,144.00 m²",113.94 m²,86.00 m²
4,E,35.00 m²,"290,000.00 m²",695.41 m²,237.00 m²
5,L,103.00 m²,"80,000.00 m²",930.54 m²,700.00 m²
6,S,15.00 m²,"2,500,000.00 m²",553.95 m²,250.00 m²
7,general,14.00 m²,"2,500,000.00 m²",421.33 m²,189.00 m²


# M2 Construcción

In [57]:
df_total_terreno = df_total[~df_total['metros_construido'].isin([0, 1,2,3])]
df_descripcion = df_total_terreno.groupby('segmento')['metros_construido'].describe().reset_index()
df_descripcion = df_descripcion[['segmento', 'min', 'max', 'mean', '50%']] #Columnas deseadas
df_descripcion.rename(columns={
    '50%': 'Mediana',
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)


# Función para formatear valores en formato metros
def formato_metros(valor):
    return f"{valor:,.2f} m²"

# Aplicar formato en m2 a los valores, excluyendo la columna 'segmento'
for i in range(1, 5):  # 1, 2, 3, 4 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    df_descripcion.iloc[:, i] = df_descripcion.iloc[:, i].astype(float).apply(formato_metros)
df_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_5124\2035657892.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     51.00 m²
1     15.00 m²
2     42.00 m²
3     42.00 m²
4     35.00 m²
5    165.00 m²
6     52.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_descripcion.iloc[:, i] = df_descripcion.iloc[:, i].astype(float).apply(formato_metros)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_5124\2035657892.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     22,415.00 m²
1        543.00 m²
2     11,267.00 m²
3      1,064.00 m²
4    280,000.00 m²
5     80,000.00 m²
6     15,683.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_descripcion.iloc[:, i] = df_descripcion.iloc[:, i].astype(f

,segmento,Mínimo,Máximo,Promedio,Mediana
0,A,51.00 m²,"22,415.00 m²",177.88 m²,166.00 m²
1,B,15.00 m²,543.00 m²,136.28 m²,132.00 m²
2,C,42.00 m²,"11,267.00 m²",128.81 m²,112.00 m²
3,D,42.00 m²,"1,064.00 m²",112.30 m²,84.00 m²
4,E,35.00 m²,"280,000.00 m²",502.63 m²,260.00 m²
5,L,165.00 m²,"80,000.00 m²",635.97 m²,500.00 m²
6,S,52.00 m²,"15,683.00 m²",276.03 m²,265.00 m²


In [58]:
todo_descripcion = df_total[~df_total['metros_construido'].isin([0,1,2,3]) & (df_total['metros_total'] > 3)]['metros_total'].describe()[['min', 'max', 'mean', '50%']].to_frame().T
todo_descripcion['segmento'] = 'general'

todo_descripcion.rename(columns={
    '50%': 'Mediana',
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Aplicar formato monetario a los valores
for i in range(4):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_metros)

todo_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_5124\4114240182.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_total    14.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_metros)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_5124\4114240182.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_total    2,500,000.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_metros)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_5124\4114240182.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will ra

,Mínimo,Máximo,Promedio,Mediana,segmento
metros_total,14.00 m²,"2,500,000.00 m²",419.74 m²,189.00 m²,general


In [59]:
# Guardar el JSON en un archivo
resultados_finales_descripcion = pd.concat([df_descripcion, todo_descripcion], ignore_index=True)
resultados_finales_desc_json = resultados_finales_descripcion.to_json(orient='index')
with open('assets/tablas/grupo_construidos.json', 'w') as f:
    f.write(resultados_finales_desc_json)

In [60]:
resultados_finales_descripcion

,segmento,Mínimo,Máximo,Promedio,Mediana
0,A,51.00 m²,"22,415.00 m²",177.88 m²,166.00 m²
1,B,15.00 m²,543.00 m²,136.28 m²,132.00 m²
2,C,42.00 m²,"11,267.00 m²",128.81 m²,112.00 m²
3,D,42.00 m²,"1,064.00 m²",112.30 m²,84.00 m²
4,E,35.00 m²,"280,000.00 m²",502.63 m²,260.00 m²
5,L,165.00 m²,"80,000.00 m²",635.97 m²,500.00 m²
6,S,52.00 m²,"15,683.00 m²",276.03 m²,265.00 m²
7,general,14.00 m²,"2,500,000.00 m²",419.74 m²,189.00 m²


# Pecio M2 Construcción

In [61]:
df_total_terreno = df_total[~df_total['metros_construido'].isin([0, 1,2,3])]
df_total_terreno = df_total_terreno[df_total_terreno['precio_m2_construido'] > 10]
grupo_filtrado = df_total_terreno.groupby('segmento')['precio_m2_construido'].describe().reset_index()
# Mantener solo las columnas que deseas (en este caso, min y max)
grupo_filtrado = grupo_filtrado[['segmento', 'min', 'max', 'mean', '50%']] #Columnas deseadas
grupo_filtrado.rename(columns={
    '50%': 'Mediana',
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Aplicar formato monetario a los valores, excluyendo la columna 'segmento'
for i in range(1, 5):  # 1, 2, 3, 4 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    grupo_filtrado.iloc[:, i] = grupo_filtrado.iloc[:, i].astype(float).apply(formato_monetario)

grupo_filtrado 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_5124\2452427253.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0      $169.53
1    $5,930.02
2      $164.11
3    $1,221.80
4      $651.04
5      $237.50
6      $464.20
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_filtrado.iloc[:, i] = grupo_filtrado.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_5124\2452427253.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0        $69,835.68
1       $175,333.33
2        $56,808.81
3        $28,571.43
4    $38,695,652.17
5        $74,733.33
6       $111,320.75
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_filtrado.iloc[:, i] = grupo_filtrado.iloc[:, i

,segmento,Mínimo,Máximo,Promedio,Mediana
0,A,$169.53,"$69,835.68","$23,622.36","$22,000.00"
1,B,"$5,930.02","$175,333.33","$23,060.99","$21,598.64"
2,C,$164.11,"$56,808.81","$19,121.77","$18,873.87"
3,D,"$1,221.80","$28,571.43","$15,801.09","$16,875.00"
4,E,$651.04,"$38,695,652.17","$80,018.90","$8,214.29"
5,L,$237.50,"$74,733.33","$32,173.26","$31,926.41"
6,S,$464.20,"$111,320.75","$24,339.18","$22,302.56"


In [66]:
todo_descripcion = df_total[(df_total['precio_m2_construido'] > 6) & (~df_total['metros_construido'].isin([0, 1,2,3]))]['precio_m2_construido'].describe()[['min', 'max', 'mean', '50%']].to_frame().T
todo_descripcion['segmento'] = 'general'
todo_descripcion = todo_descripcion[['segmento', 'min', 'max', 'mean', '50%']] #Columnas deseadas

todo_descripcion.rename(columns={
    '50%': 'Mediana',
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Aplicar formato monetario a los valores, excluyendo la columna 'segmento'
for i in range(1, 5):  # 1, 2, 3, 4 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)

todo_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_5124\3775032291.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio_m2_construido    $164.11
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_5124\3775032291.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio_m2_construido    $38,695,652.17
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_5124\3775032291.py:14: FutureWarning: Setting an item of incompatible dtype is de

,segmento,Mínimo,Máximo,Promedio,Mediana
precio_m2_construido,general,$164.11,"$38,695,652.17","$27,623.68","$21,428.57"


In [68]:
# Guardar el JSON en un archivo
resultados_finales_descripcion = pd.concat([grupo_filtrado, todo_descripcion], ignore_index=True)
resultados_finales_desc_json = resultados_finales_descripcion.to_json(orient='index')
with open('assets/tablas/grupo_precio_construidos.json', 'w') as f:
    f.write(resultados_finales_desc_json)

In [69]:
resultados_finales_descripcion

,segmento,Mínimo,Máximo,Promedio,Mediana
0,A,$169.53,"$69,835.68","$23,622.36","$22,000.00"
1,B,"$5,930.02","$175,333.33","$23,060.99","$21,598.64"
2,C,$164.11,"$56,808.81","$19,121.77","$18,873.87"
3,D,"$1,221.80","$28,571.43","$15,801.09","$16,875.00"
4,E,$651.04,"$38,695,652.17","$80,018.90","$8,214.29"
5,L,$237.50,"$74,733.33","$32,173.26","$31,926.41"
6,S,$464.20,"$111,320.75","$24,339.18","$22,302.56"
7,general,$164.11,"$38,695,652.17","$27,623.68","$21,428.57"
